In [1]:
import numpy as np

from hybrid import *
from Recommenders.KNN.UserKNN_CFCBF_Hybrid_Recommender import UserKNN_CFCBF_Hybrid_Recommender
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

from k_fold_hyperparam_search.Utility import Utility
from k_fold_hyperparam_search.evaluate import evaluate_algorithm
from k_fold_hyperparam_search.hyperparam_def import names, spaces


# Set parameters
n_calls = 25
k = 10
validation_percentage = 0.1
recommender_class = Hybrid_006022

utility = Utility()


print("Using randomized datasets. k={}, val_percentage={}".format(k, validation_percentage))

URM_aug_trains, URM_pow_trains, ICM, UCM, URM_tests = utility.give_me_randomized_k_folds_with_val_percentage(k, validation_percentage)

assert len(URM_aug_trains) == len(URM_tests)

print("Initalizing all k recommenders of type {}...".format(recommender_class.__name__))

recommenders = []
if(recommender_class == HybridRecommender_7):
    for URM_train_aug, URM_train_pow in zip(URM_aug_trains, URM_pow_trains):
        recommenders.append(recommender_class(
            URM_train_aug, URM_train_pow, UCM))

elif(recommender_class == UserKNN_CFCBF_Hybrid_Recommender):
    for URM_train_aug in URM_aug_trains:
        recommenders.append(recommender_class(URM_train_aug, UCM))

elif(recommender_class == ItemKNN_CFCBF_Hybrid_Recommender):
    for URM_train_pow in URM_pow_trains:
        recommenders.append(recommender_class(URM_train_pow,ICM))

elif(recommender_class == UserKNNCFRecommender):
    for URM_train_aug in URM_aug_trains:
        recommenders.append(recommender_class(URM_train_aug))

elif(recommender_class == SLIMElasticNetRecommender or recommender_class == RP3betaRecommender):
    for URM_train_pow in URM_pow_trains:
        recommenders.append(recommender_class(URM_train_pow))

elif(recommender_class==Hybrid_User_and_Item_KNN_CFCBF_Hybrid):
    for URM_train_aug, URM_train_pow in zip(URM_aug_trains, URM_pow_trains):
        _ItemKNN_CFCBF_Hybrid_Recommender = ItemKNN_CFCBF_Hybrid_Recommender(URM_train_aug,ICM)
        _ItemKNN_CFCBF_Hybrid_Recommender.fit(0.011278462705558101,topK=661,shrink=36)

        _UserKNN_CFCBF_Hybrid_Recommender = UserKNN_CFCBF_Hybrid_Recommender(URM_train_aug,UCM)
        _UserKNN_CFCBF_Hybrid_Recommender.fit(0.01,topK=669,shrink=50)

        recommenders.append(recommender_class(URM_train_aug,URM_train_pow,_ItemKNN_CFCBF_Hybrid_Recommender,_UserKNN_CFCBF_Hybrid_Recommender))
elif(recommender_class == Hybrid_006022):
    for URM_train_aug, URM_train_pow in zip(URM_aug_trains, URM_pow_trains):
        UserKNNCF = UserKNNCFRecommender(URM_train_aug)
        UserKNNCF.fit()

        RP3beta_aug = RP3betaRecommender(URM_train_aug)
        RP3beta_aug.fit()

        S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
        S_SLIM.fit()

        Hybrid_SSLIM_RP3B_aug = Hybrid_SSLIM_RP3B_aug(
            URM_train_aug, S_SLIM, RP3beta_aug)
        Hybrid_SSLIM_RP3B_aug.fit(alpha = 0.7447123958484749)

        Hybrid_006022 = Hybrid_006022(URM_train_aug, URM_train_pow, ICM, UCM, Hybrid_SSLIM_RP3B_aug, UserKNNCF)
        Hybrid_006022.fit(Hybrid_1_tier1_weight= 0.4730071105820606, Hybrid_2_tier1_weight= 1.0, Hybrid_1_tier2_weight= 1.0, Hybrid_2_tier2_weight= 1.0, Hybrid_1_tier3_weight=1.0)

else:
    for URM_train_aug, URM_train_pow in zip(URM_aug_trains, URM_pow_trains):
        recommenders.append(recommender_class(
            URM_train_aug, URM_train_pow))

print("Finished initialization!\nStarting fitting and evaluating...")

Using randomized datasets. k=10, val_percentage=0.1
Initalizing all k recommenders of type Hybrid_006022...
UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 41629 (100.0%), 1375.09 column/sec. Elapsed time 30.27 sec
RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 2639.50 column/sec. Elapsed time 10.60 sec
SLIMElasticNetRecommender: Processed 2877 (10.3%) in 5.00 min. Items per second: 9.58
SLIMElasticNetRecommender: Processed 5358 (19.2%) in 10.00 min. Items per second: 8.93
SLIMElasticNetRecommender: Processed 8808 (31.5%) in 15.00 min. Items per second: 9.78
SLIMElasticNetRecommender: Processed 12265 (43.9%) in 20.00 min. Items per second: 10.22
SLIMElasticNetRecommender: Processed 15783 (56.4%) in 25.00 min. Items per second: 10.52
SLIMElasticNetRecommender: Processed 19278 (68.9%) in 30.01 min. Items per second: 10.71
SLIMElasticNetRecommender: Processed 2284

In [ ]:
scores = []
count=1
for recommender, test in zip(recommenders, URM_tests):
    recommender.fit(ItemKNN_CFCBF_Hybrid_Recommender_weight=0.1643020180322596, UserKNN_CFCBF_Hybrid_Recommender_weight= 0.1181324258536363)
    _, _, MAP = evaluate_algorithm(test, recommender)
    scores.append(MAP)
    print("Evaluated fold number: {}".format(count))
    count=count+1

print("Finished fit & evaluation!")

print(">>> Average MAP: {}, DIFF (= max_map - min_map): {}".format(sum(scores) / len(scores), max(scores) - min(scores)))
